[GDCV12] 직접 만들어보는 OCR

1) End-to-End OCR

2) 루브릭

3) 회고

1) End-to-End OCR

In [3]:
pip install lmdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.9/305.9 kB 4.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import six
import math
import lmdb
import os
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from IPython.display import display

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence, plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

import keras_ocr

2022-12-19 14:05:27.168322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 14:05:27.459384: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-19 14:05:28.297856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.7/lib64
2022-12-19 14:05:28.297940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

In [ ]:
BATCH_SIZE = 128
DATA_DIR = '/home/june/Github/GDCV/data/gd12'

# MJ 데이터셋 위치
TRAIN_DATA_PATH = os.path.join(DATA_DIR, '/MJ_train')
VALID_DATA_PATH = os.path.join(DATA_DIR, '/MJ_valid')
TEST_DATA_PATH = os.path.join(DATA_DIR, '/MJ_test')

2) 루브릭

1. Text recognition을 위해 특화된 데이터셋 구성이 체계적으로 진행되었다.

텍스트 이미지 리사이징, ctc loss 측정을 위한 라벨 인코딩, 배치처리 등이 적절히 수행되었다.


2. CRNN 기반의 recognition 모델의 학습이 정상적으로 진행되었다.

학습결과 loss가 안정적으로 감소하고 대부분의 문자인식 추론 결과가 정확하다.


3. keras-ocr detector와 CRNN recognizer를 엮어 원본 이미지 입력으로부터 text가 출력되는 OCR이 End-to-End로 구성되었다.

샘플 이미지를 원본으로 받아 OCR 수행 결과를 리턴하는 1개의 함수가 만들어졌다.

3) 회고

- 이번 프로젝트에서 **어려웠던 점,**


- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**


- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**


- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**


- **자기 다짐**